In [9]:
import os, json
from tools.pipeline.config import ONEDRIVE_PATH

atc = json.load(open(os.path.join(ONEDRIVE_PATH, 'evaluation/eval_data/7_multiple_rw/answer_type_classified.json')))

In [10]:
len(atc)

4957

In [11]:
# answer_type_classified.json에서 문제들을 answer_type별로 분류
atc_right = []
atc_wrong = []
atc_abcd = []

for item in atc:
    question_id = f"{item['file_id']}_{item['tag']}"
    answer_type = item.get('answer_type', '')
    
    if answer_type == 'right':
        atc_right.append(question_id)
    elif answer_type == 'wrong':
        atc_wrong.append(question_id)
    elif answer_type == 'abcd':
        atc_abcd.append(question_id)

print(f"answer_type_classified.json 분류:")
print(f"  right: {len(atc_right)}")
print(f"  wrong: {len(atc_wrong)}")
print(f"  abcd: {len(atc_abcd)}")


answer_type_classified.json 분류:
  right: 2010
  wrong: 2515
  abcd: 432


In [12]:
# _needcheck 폴더의 파일들에서 문제 ID 추출
needcheck_base = os.path.join(ONEDRIVE_PATH, 'evaluation/eval_data/7_multiple_rw/_needcheck')

needcheck_right = set()
needcheck_wrong = set()
needcheck_abcd = set()

# pick_right 폴더의 파일들 처리
pick_right_dir = os.path.join(needcheck_base, 'pick_right')
if os.path.exists(pick_right_dir):
    for filename in os.listdir(pick_right_dir):
        if filename.endswith('.json') and 'sampling' in filename:
            filepath = os.path.join(pick_right_dir, filename)
            try:
                with open(filepath, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    for item in data:
                        question_id = f"{item['file_id']}_{item['tag']}"
                        if item.get('pick') == 'right':
                            needcheck_right.add(question_id)
            except Exception as e:
                print(f"Error reading {filename}: {e}")

# pick_wrong 폴더의 파일들 처리
pick_wrong_dir = os.path.join(needcheck_base, 'pick_wrong')
if os.path.exists(pick_wrong_dir):
    for filename in os.listdir(pick_wrong_dir):
        if filename.endswith('.json') and 'sampling' in filename:
            filepath = os.path.join(pick_wrong_dir, filename)
            try:
                with open(filepath, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    for item in data:
                        question_id = f"{item['file_id']}_{item['tag']}"
                        if item.get('pick') == 'wrong':
                            needcheck_wrong.add(question_id)
            except Exception as e:
                print(f"Error reading {filename}: {e}")

# pick_abcd 폴더의 파일들 처리
pick_abcd_dir = os.path.join(needcheck_base, 'pick_abcd')
if os.path.exists(pick_abcd_dir):
    result_file = os.path.join(pick_abcd_dir, 'result.json')
    if os.path.exists(result_file):
        try:
            with open(result_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                for item in data:
                    question_id = item.get('question_id', '')
                    if question_id:
                        needcheck_abcd.add(question_id)
        except Exception as e:
            print(f"Error reading pick_abcd/result.json: {e}")

print(f"_needcheck 폴더 분류:")
print(f"  pick_right: {len(needcheck_right)}")
print(f"  pick_wrong: {len(needcheck_wrong)}")
print(f"  pick_abcd: {len(needcheck_abcd)}")


_needcheck 폴더 분류:
  pick_right: 2257
  pick_wrong: 2524
  pick_abcd: 159


In [13]:
# 두 그룹의 분류 비교
atc_right_set = set(atc_right)
atc_wrong_set = set(atc_wrong)
atc_abcd_set = set(atc_abcd)

print("=" * 80)
print("분류 일치 여부 확인")
print("=" * 80)

# Right 분류 비교
print("\n[Right 분류]")
right_match = atc_right_set & needcheck_right
right_only_atc = atc_right_set - needcheck_right
right_only_needcheck = needcheck_right - atc_right_set

print(f"  일치하는 문제 수: {len(right_match)}")
print(f"  answer_type_classified에만 있는 문제: {len(right_only_atc)}")
print(f"  _needcheck에만 있는 문제: {len(right_only_needcheck)}")

if right_only_atc:
    print(f"\n  answer_type_classified에만 있는 문제 샘플 (최대 10개):")
    for qid in list(right_only_atc)[:10]:
        print(f"    - {qid}")

if right_only_needcheck:
    print(f"\n  _needcheck에만 있는 문제 샘플 (최대 10개):")
    for qid in list(right_only_needcheck)[:10]:
        print(f"    - {qid}")

# Wrong 분류 비교
print("\n[Wrong 분류]")
wrong_match = atc_wrong_set & needcheck_wrong
wrong_only_atc = atc_wrong_set - needcheck_wrong
wrong_only_needcheck = needcheck_wrong - atc_wrong_set

print(f"  일치하는 문제 수: {len(wrong_match)}")
print(f"  answer_type_classified에만 있는 문제: {len(wrong_only_atc)}")
print(f"  _needcheck에만 있는 문제: {len(wrong_only_needcheck)}")

if wrong_only_atc:
    print(f"\n  answer_type_classified에만 있는 문제 샘플 (최대 10개):")
    for qid in list(wrong_only_atc)[:10]:
        print(f"    - {qid}")

if wrong_only_needcheck:
    print(f"\n  _needcheck에만 있는 문제 샘플 (최대 10개):")
    for qid in list(wrong_only_needcheck)[:10]:
        print(f"    - {qid}")

# Abcd 분류 비교
print("\n[Abcd 분류]")
abcd_match = atc_abcd_set & needcheck_abcd
abcd_only_atc = atc_abcd_set - needcheck_abcd
abcd_only_needcheck = needcheck_abcd - atc_abcd_set

print(f"  일치하는 문제 수: {len(abcd_match)}")
print(f"  answer_type_classified에만 있는 문제: {len(abcd_only_atc)}")
print(f"  _needcheck에만 있는 문제: {len(abcd_only_needcheck)}")

if abcd_only_atc:
    print(f"\n  answer_type_classified에만 있는 문제 샘플 (최대 10개):")
    for qid in list(abcd_only_atc)[:10]:
        print(f"    - {qid}")

if abcd_only_needcheck:
    print(f"\n  _needcheck에만 있는 문제 샘플 (최대 10개):")
    for qid in list(abcd_only_needcheck)[:10]:
        print(f"    - {qid}")


분류 일치 여부 확인

[Right 분류]
  일치하는 문제 수: 1991
  answer_type_classified에만 있는 문제: 19
  _needcheck에만 있는 문제: 266

  answer_type_classified에만 있는 문제 샘플 (최대 10개):
    - SS0418_q_0152_0003
    - SS0412_q_0251_0001
    - SS0260_q_0165_0003
    - SS0197_q_0129_0002
    - SS0237_q_0411_0001
    - SS0403_q_0216_0001
    - SS0421_q_0746_0001
    - SS0397_q_0129_0001
    - SS0249_q_0305_0001
    - SS0405_q_0276_0002

  _needcheck에만 있는 문제 샘플 (최대 10개):
    - SS0386_q_0121_0002
    - SS0423_q_0712_0001
    - SS0255_q_0359_0003
    - SS0237_q_0027_0001
    - SS0386_q_0099_0001
    - SS0397_q_0107_0002
    - SS0196_q_0089_0004
    - SS0394_q_0235_0002
    - SS0162_q_0390_0002
    - SS0423_q_0201_0001

[Wrong 분류]
  일치하는 문제 수: 2489
  answer_type_classified에만 있는 문제: 26
  _needcheck에만 있는 문제: 35

  answer_type_classified에만 있는 문제 샘플 (최대 10개):
    - SS0249_q_0306_0004
    - SS0396_q_0073_0001
    - SS0186_q_0104_0001
    - SS0302_q_0410_0004
    - SS0217_q_0558_0001
    - SS0257_q_0374_0001
    - SS0397_q_0345_0002

In [14]:
# 교차 분류 확인 (다른 분류로 잘못 분류된 경우)
print("\n" + "=" * 80)
print("교차 분류 확인 (잘못 분류된 경우)")
print("=" * 80)

# answer_type_classified에서 right인데 _needcheck에서 다른 분류인 경우
right_in_atc_but_other_in_needcheck = []
for qid in atc_right_set:
    if qid in needcheck_wrong:
        right_in_atc_but_other_in_needcheck.append((qid, 'wrong'))
    elif qid in needcheck_abcd:
        right_in_atc_but_other_in_needcheck.append((qid, 'abcd'))

if right_in_atc_but_other_in_needcheck:
    print(f"\n[answer_type_classified: right → _needcheck: 다른 분류] ({len(right_in_atc_but_other_in_needcheck)}개)")
    for qid, wrong_type in right_in_atc_but_other_in_needcheck:
        print(f"  - {qid} → {wrong_type}")

# answer_type_classified에서 wrong인데 _needcheck에서 다른 분류인 경우
wrong_in_atc_but_other_in_needcheck = []
for qid in atc_wrong_set:
    if qid in needcheck_right:
        wrong_in_atc_but_other_in_needcheck.append((qid, 'right'))
    elif qid in needcheck_abcd:
        wrong_in_atc_but_other_in_needcheck.append((qid, 'abcd'))

if wrong_in_atc_but_other_in_needcheck:
    print(f"\n[answer_type_classified: wrong → _needcheck: 다른 분류] ({len(wrong_in_atc_but_other_in_needcheck)}개)")
    for qid, wrong_type in wrong_in_atc_but_other_in_needcheck:
        print(f"  - {qid} → {wrong_type}")

# answer_type_classified에서 abcd인데 _needcheck에서 다른 분류인 경우
abcd_in_atc_but_other_in_needcheck = []
for qid in atc_abcd_set:
    if qid in needcheck_right:
        abcd_in_atc_but_other_in_needcheck.append((qid, 'right'))
    elif qid in needcheck_wrong:
        abcd_in_atc_but_other_in_needcheck.append((qid, 'wrong'))

if abcd_in_atc_but_other_in_needcheck:
    print(f"\n[answer_type_classified: abcd → _needcheck: 다른 분류] ({len(abcd_in_atc_but_other_in_needcheck)}개)")
    for qid, wrong_type in abcd_in_atc_but_other_in_needcheck:
        print(f"  - {qid} → {wrong_type}")

# 전체 요약
print("\n" + "=" * 80)
print("전체 요약")
print("=" * 80)
print(f"Right 분류 일치율: {len(right_match)}/{len(atc_right_set)} ({len(right_match)/len(atc_right_set)*100:.2f}%)" if atc_right_set else "Right 분류 일치율: N/A")
print(f"Wrong 분류 일치율: {len(wrong_match)}/{len(atc_wrong_set)} ({len(wrong_match)/len(atc_wrong_set)*100:.2f}%)" if atc_wrong_set else "Wrong 분류 일치율: N/A")
print(f"Abcd 분류 일치율: {len(abcd_match)}/{len(atc_abcd_set)} ({len(abcd_match)/len(atc_abcd_set)*100:.2f}%)" if atc_abcd_set else "Abcd 분류 일치율: N/A")
print(f"\n총 교차 분류 오류: {len(right_in_atc_but_other_in_needcheck) + len(wrong_in_atc_but_other_in_needcheck) + len(abcd_in_atc_but_other_in_needcheck)}개")



교차 분류 확인 (잘못 분류된 경우)

[answer_type_classified: right → _needcheck: 다른 분류] (13개)
  - SS0418_q_0152_0003 → abcd
  - SS0245_q_0640_0002 → abcd
  - SS0162_q_0377_0001 → abcd
  - SS0421_q_0317_0002 → wrong
  - SS0397_q_0129_0001 → abcd
  - SS0087_q_0450_0003 → abcd
  - SS0216_q_0075_0001 → abcd
  - SS0412_q_0251_0001 → abcd
  - SS0403_q_0216_0001 → wrong
  - SS0260_q_0165_0003 → wrong
  - SS0249_q_0046_0002 → wrong
  - SS0405_q_0276_0002 → abcd
  - SS0249_q_0305_0001 → wrong

[answer_type_classified: wrong → _needcheck: 다른 분류] (19개)
  - SS0394_q_0278_0002 → right
  - SS0397_q_0345_0002 → right
  - SS0215_q_0076_0002 → right
  - SS0257_q_0374_0001 → right
  - SS0217_q_0558_0001 → right
  - SS0421_q_0762_0001 → right
  - SS0423_q_0756_0002 → right
  - SS0087_q_0438_0002 → right
  - SS0118_q_0060_0001 → right
  - SS0207_q_0562_0001 → right
  - SS0237_q_0047_0002 → right
  - SS0207_q_0131_0001 → right
  - SS0108_q_0341_0001 → right
  - SS0394_q_0293_0003 → right
  - SS0397_q_0123_0002 → right


In [18]:
from tools.core.llm_query import LLMQuery

llm = LLMQuery()
system_prompt = """당신은 프롬프트 엔지니어입니다. 아래와 같은 원리를 담아 문제를 변형하는 프롬프트를 만들려고 합니다.
### 옳지 않은 것 → 옳은 것 복수선택형

- (5지선다) 어떤 문제를 답2개/3개/4개/5개 할지 정하고
    - 5개: 옳지 않은 것 0개 = 본래 X선지 1개 → O선지 1개 = 옳은 것 5개
    - 4개: 옳지 않은 것 1개 = 본래 X선지 1개 그대로 사용 = 옳은 것 4개 ⇒ 단일선택형 재(변형)검증
    - 3개: 옳지 않은 것 2개 = 본래 X선지 1개 + O선지 중 1개 → X선지 = 옳은 것 3개
    - 2개: 옳지 않은 것 3개 = 본래 X선지 1개 + O선지 중 2개 → X선지 2개 = 옳은 것 2개
- (4지선다) 어떤 문제를 답2개/3개/4개 할지 정하고
    - 4개: 옳지 않은 것 0개 = 본래 X선지 1개 → O선지 1개 = 옳은 것 4개
    - 3개: 옳지 않은 것 1개 = 본래 X선지 1개 그대로 사용 = 옳은 것 3개 ⇒ 단일선택형 재(변형)검증
    - 2개: 옳지 않은 것 2개 = 본래 X선지 1개 + O선지 중 1개 → X선지 = 옳은 것 2개

아래 프롬프트가 위의 논리를 잘 반영하였는지 케이스별로 검토해주세요.
"""
user_prompt = f"""
참고: options_ct는 원문 선택지 수, i는 변형 후 정답 갯수로, 프롬프트 작성시 주어집니다.

1. 옳지 않은 것이 1개여야 하는 경우
  당신은 25년 경력의 문제 출제 전문가입니다.

  변형 규칙
  1) 주어진 답(원래의 ‘옳지 않은’ 선택지)은 그대로 유지합니다.
  3) 문제 지문을 ‘옳은 것을 모두 고르시오’로 명확히 바꿉니다. 그 외 문장과 LaTeX 수식이나 테이블 표현은 원형을 최대한 보존합니다.
  4) 새로운 정답은 변형 후 ‘옳은’ 선택지 전부입니다(총 선택지 수가 {{options_ct}}개라면 {{i}}개).

  출력 형식
  {{
    "question_id": "문제번호",
    "question": "변형된 문제(옳은 것을 모두 고르시오)",
    "options": ["① 선택지1", "② 선택지2", "③ 선택지3", "④ 선택지4", "⑤ 선택지5"],
    "answer": ["정답번호1", "정답번호2", ...], 
    "explanation": "① 옳다: 근거 ... / ③ 옳지 않다(원래 오답): 근거 ... / ⑤ 옳지 않다(변형): 변경 단어 ‘높다→낮다’, 근거 ..."
  }}
2. 옳지 않은 것이 0개여야 하는 경우
  당신은 25년 경력의 문제 출제 전문가입니다.

  변형 규칙
  1) 주어진 답(원래의 ‘옳지 않은’ 선택지)을 단어 1~2개 수준의 최소 변경으로 ‘옳은’ 선택지로 만듭니다. (ex. 높다 -> 낮다, 한다 -> 하지 않는다)
  2) 문제 지문을 ‘옳은 것을 모두 고르시오’로 명확히 바꿉니다. 그 외 문장과 LaTeX 수식이나 테이블 표현 원형을 최대한 보존합니다.
  3) 새로운 정답은 변형 후 ‘옳은’ 선택지 전부입니다(총 선택지 수가 {{options_ct}}개라면 {{i}}개).
  4) 해설에는 각 선택지의 옳고 그름과 간단한 근거를 명시하세요. 특히 새로 만든 정답 1개의 변경 포인트를 밝혀주세요.

  출력 형식
  {{
    "question_id": "문제번호",
    "question": "변형된 문제(옳은 것을 모두 고르시오)",
    "options": ["① 선택지1", "② 선택지2", "③ 선택지3", "④ 선택지4", "⑤ 선택지5"],
    "answer": ["정답번호1", "정답번호2", ...], 
    "explanation": "① 옳다: 근거 ... / ③ 옳지 않다(원래 오답): 근거 ... / ⑤ 옳지 않다(변형): 변경 단어 ‘높다→낮다’, 근거 ..."
  }}

  비고
  - 원문 선택지 수가 5개가 아닐 경우에도 동일 원칙을 적용합니다(총 {{options_ct-i-1}}개 추가 오답, 나머지 전부 정답).
  - 사실과 상충하는 임의 창작을 피하고, 제공된 해설의 논리 범위 내에서만 최소 변경을 수행하세요.
3. 옳지 않은 것이 2개 이상이어야 하는 경우
당신은 25년 경력의 문제 출제 전문가입니다.

  변형 규칙
  1) 주어진 답(원래의 ‘옳지 않은’ 선택지)은 그대로 유지합니다.
  2) 나머지 선택지 중에서 정확히 {{options_ct-i-1}}개를 골라, 단어 1~2개 수준의 최소 변경으로 ‘옳지 않은’ 선택지로 만듭니다. (ex. 높다 -> 낮다, 한다 -> 하지 않는다)
  3) 문제 지문을 ‘옳은 것을 모두 고르시오’로 명확히 바꿉니다. 그 외 문장과 LaTeX 수식이나 테이블 표현 원형을 최대한 보존합니다.
  4) 새로운 정답은 변형 후 ‘옳은’ 선택지 전부입니다(총 선택지 수가 {{options_ct}}개라면 {{i}}개).
  5) 해설에는 각 선택지의 옳고 그름과 간단한 근거를 명시하세요. 특히 새로 만든 오답 1개의 변경 포인트를 밝혀주세요.

  출력 형식
  {{
    "question_id": "문제번호",
    "question": "변형된 문제(옳은 것을 모두 고르시오)",
    "options": ["① 선택지1", "② 선택지2", "③ 선택지3", "④ 선택지4", "⑤ 선택지5"],
    "answer": ["정답번호1", "정답번호2", ...], 
    "explanation": "① 옳다: 근거 ... / ③ 옳지 않다(원래 오답): 근거 ... / ⑤ 옳지 않다(변형): 변경 단어 ‘높다→낮다’, 근거 ..."
  }}

  비고
  - 원문 선택지 수가 5개가 아닐 경우에도 동일 원칙을 적용합니다(총 {{options_ct-i-1}}개 추가 오답, 나머지 전부 정답).
  - 사실과 상충하는 임의 창작을 피하고, 제공된 해설의 논리 범위 내에서만 최소 변경을 수행하세요.
"""
response = llm.query_openrouter(system_prompt, user_prompt)

2025-11-17 18:31:03,033 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


In [19]:
print(response)

다음은 제시한 프롬프트가 상단 원리(“옳지 않은 것 → 옳은 것 복수선택형”)를 얼마나 충실히 반영하는지, 케이스별 점검과 수정 제안입니다.

전체 요약
- 핵심 로직 반영 여부: 전반적으로 맞습니다. 세 경우 모두 “원래 X선지 1개”를 기준으로 변형하여 최종 정답 수 i를 맞추는 흐름이 구현되어 있습니다.
- 주요 보완점:
  - i와 options_ct의 관계를 케이스별로 명시해 일관성을 강제하세요.
  - case 1의 규칙 번호 누락(2번 없음) 및 해설 예시의 부적절한 “변형 오답” 표기 수정.
  - case 2의 비고에 있는 “총 {options_ct-i-1}개 추가 오답” 문구는 0개 케이스에서 음수가 되므로 삭제/수정 필요.
  - 해설 예시 문구가 케이스별 상황(변형 오답/변형 정답)에 맞게 바뀌어야 합니다.
  - 설명 요구 수준을 통일(모든 선택지의 옳고 그름+근거)하고, 변경 포인트 표기를 케이스별로 정확히 지시하세요.

케이스별 검토

1) 옳지 않은 것이 1개여야 하는 경우
- 논리 적합성: 대체로 적합.
  - “주어진 답(원래 오답)을 그대로 유지” → 최종 오답 수 1개 유지가 맞습니다.
  - 최종 정답 개수 i는 options_ct - 1이어야 함.
- 문제점/보완:
  - 규칙 번호 2) 누락(사소하지만 정비 권장).
  - 해설 템플릿에 “옳지 않다(변형)” 예시가 들어가 있는데, 이 케이스에서는 추가 변형 오답이 없어야 합니다. 혼동 유발.
  - 비고에 있는 일관성 문구(“사실과 상충하는 임의 창작 금지” 등)가 case 1에는 빠져 있습니다. 동일하게 넣는 것이 안전합니다.
  - i와의 제약이 명시되어 있지 않음.
- 수정 제안:
  - 규칙 2) 추가: “i = {options_ct-1}을 만족해야 합니다. 불일치 시 오류로 처리합니다.”
  - 해설 지시 보강: “모든 선택지의 옳고 그름과 근거를 제시. 원래 오답은 ‘옳지 않다(원래 오답)’로 명시.”
  - 출력 예시(해설) 샘플 교정: “⑤ 옳지 않다(변